In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import xgboost as xgb
import itertools
import warnings
from matplotlib.lines import Line2D
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet

# Baseline/default

### Lag only

In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Load the dataset
df_lagged = pd.read_csv(r"C:\Users\dylan\OneDrive - HvA\AAI Master\BLOK 3 afstuderen\Thesis\Eigenwerk\DATASET\df_lagged.csv")  # Pad aanpassen indien nodig
target_col = 'Totaal_verkochte_wegvoertuigen'
drop_cols = ['Periode', target_col]

# Configuration
forecast_horizon = 3
initial_train_size = 12
step_size = 3
target_col = 'Totaal_verkochte_wegvoertuigen'
drop_cols = ['Periode', target_col]

# Define feature sets
lag_time_features = ['TVV_Lag1', 'TVV_Lag2', 'TVV_Lag3', 'TVV_Lag4', 'TVV_Lag5', 'TVV_Lag6', 'year', 'month', 'quarter']
all_features = [
    'thuis opladen', 'elektrische auto', 'hybride elektrisch',
    'Bijtelling (%)', 'Consumentenvertrouwen', 'Economisch klimaat',
    'Koopbereidheid', 'Grote aankopen',
    'BenzineEuro95_1_first', 'BenzineEuro95_1_last', 'BenzineEuro95_1_min',
    'BenzineEuro95_1_max', 'BenzineEuro95_1_mean',
    'Diesel_2_first', 'Diesel_2_last', 'Diesel_2_min', 'Diesel_2_max', 'Diesel_2_mean'
] + lag_time_features

selected_feature_sets = {
    '(Lag Only)': lag_time_features,
    'All Features': all_features
}


# Prepare features and target
X = df_lagged[lag_time_features].values
y = df_lagged[target_col].values

# Containers for results
split_debug = []

# Walk-forward loop with internal validation split (90% train, 10% val inside train block)
for i in range(initial_train_size, len(df_lagged) - forecast_horizon + 1, step_size):
    X_total_train = X[:i]
    y_total_train = y[:i]
    X_test = X[i:i + forecast_horizon]
    y_test = y[i:i + forecast_horizon]

    # Split train into sub-train and validation (90/10 split)
    val_size = int(0.1 * len(X_total_train))
    if val_size < 1:
        continue  # Skip if not enough data
    X_sub_train = X_total_train[:-val_size]
    y_sub_train = y_total_train[:-val_size]
    X_val = X_total_train[-val_size:]
    y_val = y_total_train[-val_size:]

    # Store sizes and shapes for verification
    split_debug.append({
        'train_range': (0, i - val_size),
        'val_range': (i - val_size, i),
        'test_range': (i, i + forecast_horizon),
        'train_shape': X_sub_train.shape,
        'val_shape': X_val.shape,
        'test_shape': X_test.shape
    })




### linear regression 

In [10]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import pandas as pd

results_lr_val = []

for i in range(initial_train_size, len(df_lagged) - forecast_horizon + 1, step_size):
    X_total_train = X[:i]
    y_total_train = y[:i]
    X_test = X[i:i + forecast_horizon]
    y_test = y[i:i + forecast_horizon]

    val_size = int(0.1 * len(X_total_train))
    if val_size < 1:
        continue

    X_sub_train = X_total_train[:-val_size]
    y_sub_train = y_total_train[:-val_size]
    X_val = X_total_train[-val_size:]
    y_val = y_total_train[-val_size:]

    model = LinearRegression()
    model.fit(X_sub_train, y_sub_train)
    y_pred_val = model.predict(X_val)

    mae = mean_absolute_error(y_val, y_pred_val)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred_val))
    r2 = r2_score(y_val, y_pred_val)
    mape = np.mean(np.abs((y_val - y_pred_val) / y_val)) * 100

    results_lr_val.append({
        'Window': i,
        'MAE': mae,
        'RMSE': rmse,
        'R²': r2,
        'MAPE': mape
    })

results_lr_df = pd.DataFrame(results_lr_val)
print(results_lr_df)

print("Average MAPE:", results_lr_df['MAPE'].mean())



c:\Users\dylan\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
c:\Users\dylan\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
c:\Users\dylan\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


    Window          MAE         RMSE        R²       MAPE
0       12   171.435862   171.435862       NaN  47.097764
1       15   221.894082   221.894082       NaN  74.461101
2       18    16.090061    16.090061       NaN   7.965377
3       21    43.889568    44.759265 -3.767143  16.821861
4       24    60.088119    61.408863 -0.904329  12.943547
..     ...          ...          ...       ...        ...
61     195  2313.754549  3103.310312  0.438942  10.994136
62     198  2511.564198  3214.560330  0.397048  11.678466
63     201  2736.770643  3461.329876  0.359333  12.168421
64     204  2746.903514  3539.221154  0.162840  11.266740
65     207  2588.263932  3381.603244 -0.144723  10.227625

[66 rows x 5 columns]
Average MAPE: 26.626406859955992


### Arimax

In [14]:
results_arimax_val = []

for i in range(initial_train_size, len(df_lagged) - forecast_horizon + 1, step_size):
    X_total_train = X[:i]
    y_total_train = y[:i]
    X_test = X[i:i + forecast_horizon]
    y_test = y[i:i + forecast_horizon]

    val_size = int(0.1 * len(X_total_train))
    if val_size < 1:
        continue

    X_sub_train = X_total_train[:-val_size]
    y_sub_train = y_total_train[:-val_size]
    X_val = X_total_train[-val_size:]
    y_val = y_total_train[-val_size:]

    # Fit SARIMAX (ARIMAX) model
    model = SARIMAX(y_sub_train, exog=X_sub_train, order=(1, 0, 0), enforce_stationarity=False, enforce_invertibility=False)
    model_fit = model.fit(disp=False)
    y_pred_val = model_fit.predict(start=len(y_sub_train), end=len(y_sub_train)+val_size-1, exog=X_val)

    mae = mean_absolute_error(y_val, y_pred_val)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred_val))
    r2 = r2_score(y_val, y_pred_val)
    mape = np.mean(np.abs((y_val - y_pred_val) / y_val)) * 100

    results_arimax_val.append({
        'Window': i,
        'MAE': mae,
        'RMSE': rmse,
        'R²': r2,
        'MAPE': mape
    })

results_arimax_df = pd.DataFrame(results_arimax_val)
print(results_arimax_df)

print("Average MAPE:", results_arimax_df['MAPE'].mean())


c:\Users\dylan\anaconda3\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
c:\Users\dylan\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
c:\Users\dylan\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
c:\Users\dylan\anaconda3\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
c:\Users\dylan\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined wi

    Window          MAE         RMSE        R²        MAPE
0       12   460.641766   460.641766       NaN  126.549936
1       15   171.902419   171.902419       NaN   57.685375
2       18    56.228089    56.228089       NaN   27.835688
3       21    35.026311    38.284740 -2.487737   12.772542
4       24    26.290092    36.251106  0.336375    5.062614
..     ...          ...          ...       ...         ...
61     195  2376.438975  2957.004829  0.490597   11.722647
62     198  2773.258669  3243.991479  0.385957   13.251256
63     201  2888.308524  3398.939577  0.382221   13.135255
64     204  2734.545172  3358.214029  0.246281   11.571554
65     207  2563.673960  3226.777386 -0.042301   10.358943

[66 rows x 5 columns]
Average MAPE: 29.231549683121937


xgboost

In [16]:
results_xgb_val = []

for i in range(initial_train_size, len(df_lagged) - forecast_horizon + 1, step_size):
    X_total_train = X[:i]
    y_total_train = y[:i]
    X_test = X[i:i + forecast_horizon]
    y_test = y[i:i + forecast_horizon]

    val_size = int(0.1 * len(X_total_train))
    if val_size < 1:
        continue

    X_sub_train = X_total_train[:-val_size]
    y_sub_train = y_total_train[:-val_size]
    X_val = X_total_train[-val_size:]
    y_val = y_total_train[-val_size:]

    model = xgb.XGBRegressor()
    model.fit(X_sub_train, y_sub_train)
    y_pred_val = model.predict(X_val)

    mae = mean_absolute_error(y_val, y_pred_val)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred_val))
    r2 = r2_score(y_val, y_pred_val)
    mape = np.mean(np.abs((y_val - y_pred_val) / y_val)) * 100

    results_xgb_val.append({
        'Window': i,
        'MAE': mae,
        'RMSE': rmse,
        'R²': r2,
        'MAPE': mape
    })

results_xgb_df = pd.DataFrame(results_xgb_val)
print(results_xgb_df)

print("Average MAPE:", results_xgb_df['MAPE'].mean())


c:\Users\dylan\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
c:\Users\dylan\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
c:\Users\dylan\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


    Window          MAE         RMSE         R²       MAPE
0       12    77.001404    77.001405        NaN  21.154232
1       15    35.000244    35.000244        NaN  11.745048
2       18    88.847626    88.847625        NaN  43.983973
3       21    98.499245   100.609840 -23.086472  36.585089
4       24   183.060455   202.644663 -19.737209  36.962020
..     ...          ...          ...        ...        ...
61     195  6652.556641  7828.827754  -2.570677  31.420516
62     198  4911.979980  6208.320546  -1.248991  21.225461
63     201  5575.536621  6691.439905  -1.394339  23.345009
64     204  8016.132812  8923.092737  -4.321372  32.512595
65     207  8551.697266  9295.927711  -7.650485  33.797680

[66 rows x 5 columns]
Average MAPE: 30.12995589698149


baseline all features:

lr

In [15]:
# Use all features
X = df_lagged[selected_feature_sets['All Features']].values

results_lr_val = []

for i in range(initial_train_size, len(df_lagged) - forecast_horizon + 1, step_size):
    X_total_train = X[:i]
    y_total_train = y[:i]
    X_test = X[i:i + forecast_horizon]
    y_test = y[i:i + forecast_horizon]

    val_size = int(0.1 * len(X_total_train))
    if val_size < 1:
        continue

    X_sub_train = X_total_train[:-val_size]
    y_sub_train = y_total_train[:-val_size]
    X_val = X_total_train[-val_size:]
    y_val = y_total_train[-val_size:]

    model = LinearRegression()
    model.fit(X_sub_train, y_sub_train)
    y_pred_val = model.predict(X_val)

    mae = mean_absolute_error(y_val, y_pred_val)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred_val))
    r2 = r2_score(y_val, y_pred_val)
    mape = np.mean(np.abs((y_val - y_pred_val) / y_val)) * 100

    results_lr_val.append({
        'Window': i,
        'MAE': mae,
        'RMSE': rmse,
        'R²': r2,
        'MAPE': mape
    })

results_lr_df = pd.DataFrame(results_lr_val)
print(results_lr_df)

print("Average MAPE:", results_lr_df['MAPE'].mean())


c:\Users\dylan\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
c:\Users\dylan\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
c:\Users\dylan\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


    Window          MAE         RMSE         R²        MAPE
0       12   218.715833   218.715833        NaN   60.086767
1       15    94.447542    94.447542        NaN   31.693806
2       18   531.067981   531.067981        NaN  262.904941
3       21   108.067358   108.169164 -26.841923   40.655741
4       24   277.835555   287.922435 -40.863062   57.207940
..     ...          ...          ...        ...         ...
61     195  6305.044045  7156.371497  -1.983615   32.419618
62     198  5903.320886  6619.307820  -1.556611   29.204844
63     201  5976.655444  6628.864030  -1.349767   27.989508
64     204  4701.182923  5430.333183  -0.970815   20.222636
65     207  3705.010498  4474.902793  -1.004574   14.681613

[66 rows x 5 columns]
Average MAPE: 36.060768701356764


xgboost 

In [18]:
# Use all features
X = df_lagged[selected_feature_sets['All Features']].values

results_xgb_val = []

for i in range(initial_train_size, len(df_lagged) - forecast_horizon + 1, step_size):
    X_total_train = X[:i]
    y_total_train = y[:i]
    X_test = X[i:i + forecast_horizon]
    y_test = y[i:i + forecast_horizon]

    val_size = int(0.1 * len(X_total_train))
    if val_size < 1:
        continue

    X_sub_train = X_total_train[:-val_size]
    y_sub_train = y_total_train[:-val_size]
    X_val = X_total_train[-val_size:]
    y_val = y_total_train[-val_size:]

    model = xgb.XGBRegressor()
    model.fit(X_sub_train, y_sub_train)
    y_pred_val = model.predict(X_val)

    mae = mean_absolute_error(y_val, y_pred_val)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred_val))
    r2 = r2_score(y_val, y_pred_val)
    mape = np.mean(np.abs((y_val - y_pred_val) / y_val)) * 100

    results_xgb_val.append({
        'Window': i,
        'MAE': mae,
        'RMSE': rmse,
        'R²': r2,
        'MAPE': mape
    })

results_xgb_df = pd.DataFrame(results_xgb_val)
print(results_xgb_df)

print("Average MAPE:", results_xgb_df['MAPE'].mean())


c:\Users\dylan\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
c:\Users\dylan\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
c:\Users\dylan\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


    Window          MAE         RMSE         R²       MAPE
0       12    77.001404    77.001405        NaN  21.154232
1       15    35.000244    35.000244        NaN  11.745048
2       18    88.847626    88.847625        NaN  43.983973
3       21    98.499245   100.609840 -23.086472  36.585089
4       24   183.060455   202.644663 -19.737209  36.962020
..     ...          ...          ...        ...        ...
61     195  6652.556641  7828.827754  -2.570677  31.420516
62     198  4911.979980  6208.320546  -1.248991  21.225461
63     201  5575.536621  6691.439905  -1.394339  23.345009
64     204  8016.132812  8923.092737  -4.321372  32.512595
65     207  8551.697266  9295.927711  -7.650485  33.797680

[66 rows x 5 columns]
Average MAPE: 30.12995589698149


Arimax

In [19]:
# Use all features
X = df_lagged[selected_feature_sets['All Features']].values

results_arimax_val = []

for i in range(initial_train_size, len(df_lagged) - forecast_horizon + 1, step_size):
    X_total_train = X[:i]
    y_total_train = y[:i]
    X_test = X[i:i + forecast_horizon]
    y_test = y[i:i + forecast_horizon]

    val_size = int(0.1 * len(X_total_train))
    if val_size < 1:
        continue

    X_sub_train = X_total_train[:-val_size]
    y_sub_train = y_total_train[:-val_size]
    X_val = X_total_train[-val_size:]
    y_val = y_total_train[-val_size:]

    # Fit SARIMAX (ARIMAX) model
    model = SARIMAX(y_sub_train, exog=X_sub_train, order=(1, 0, 0), enforce_stationarity=False, enforce_invertibility=False)
    model_fit = model.fit(disp=False)
    y_pred_val = model_fit.predict(start=len(y_sub_train), end=len(y_sub_train)+val_size-1, exog=X_val)

    mae = mean_absolute_error(y_val, y_pred_val)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred_val))
    r2 = r2_score(y_val, y_pred_val)
    mape = np.mean(np.abs((y_val - y_pred_val) / y_val)) * 100

    results_arimax_val.append({
        'Window': i,
        'MAE': mae,
        'RMSE': rmse,
        'R²': r2,
        'MAPE': mape
    })

results_arimax_df = pd.DataFrame(results_arimax_val)
print(results_arimax_df)

print("Average MAPE:", results_arimax_df['MAPE'].mean())


c:\Users\dylan\anaconda3\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
c:\Users\dylan\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
c:\Users\dylan\anaconda3\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
c:\Users\dylan\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
c:\Users\dylan\anaconda3\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to 

    Window          MAE         RMSE         R²        MAPE
0       12   218.603170   218.603170        NaN   60.055816
1       15    94.794402    94.794402        NaN   31.810202
2       18   531.093980   531.093980        NaN  262.917812
3       21    67.950231    68.779547 -10.256695   25.339849
4       24    92.980853    93.537278  -3.418241   19.442461
..     ...          ...          ...        ...         ...
61     195  6280.969985  7146.465296  -1.975361   32.310705
62     198  5855.636963  6577.946845  -1.524760   28.963966
63     201  5947.400740  6617.053208  -1.341401   27.852398
64     204  4614.254788  5389.503255  -0.941289   19.895032
65     207  3586.909670  4374.396706  -0.915540   14.243413

[66 rows x 5 columns]
Average MAPE: 37.43207252474737


Hyperparamer optiamization All features:


In [20]:
from bayes_opt import BayesianOptimization
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_percentage_error

# Define the function to optimize
def xgb_cv_mape(max_depth, learning_rate, n_estimators, subsample, colsample_bytree):
    max_depth = int(max_depth)
    n_estimators = int(n_estimators)
    mape_scores = []
    tscv = TimeSeriesSplit(n_splits=3)
    for train_idx, val_idx in tscv.split(X):
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]
        model = xgb.XGBRegressor(
            max_depth=max_depth,
            learning_rate=learning_rate,
            n_estimators=n_estimators,
            subsample=subsample,
            colsample_bytree=colsample_bytree,
            random_state=42,
            verbosity=0
        )
        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        mape = mean_absolute_percentage_error(y_val, y_pred)
        mape_scores.append(mape)
    return -np.mean(mape_scores)  # Negative because optimizer maximizes

# Define parameter bounds
pbounds = {
    'max_depth': (2, 10),
    'learning_rate': (0.01, 0.3),
    'n_estimators': (50, 300),
    'subsample': (0.5, 1),
    'colsample_bytree': (0.5, 1)
}

optimizer = BayesianOptimization(
    f=xgb_cv_mape,
    pbounds=pbounds,
    random_state=42,
    verbose=2
)

optimizer.maximize(init_points=5, n_iter=15)

print("Best parameters:", optimizer.max['params'])
print("Best (lowest) MAPE:", -optimizer.max['target'])


|   iter    |  target   | colsam... | learni... | max_depth | n_esti... | subsample |
-------------------------------------------------------------------------------------
| 1         | -0.4341   | 0.6873    | 0.2857    | 7.856     | 199.7     | 0.578     |
| 2         | -0.4205   | 0.578     | 0.02684   | 8.929     | 200.3     | 0.854     |
| 3         | -0.4813   | 0.5103    | 0.2913    | 8.66      | 103.1     | 0.5909    |
| 4         | -0.4147   | 0.5917    | 0.09823   | 6.198     | 158.0     | 0.6456    |
| 5         | -0.4109   | 0.8059    | 0.05045   | 4.337     | 141.6     | 0.728     |
| 6         | -0.4308   | 0.6807    | 0.168     | 5.901     | 145.3     | 0.8594    |
| 7         | -0.3792   | 0.9492    | 0.06884   | 2.653     | 138.5     | 0.5282    |
| 8         | -0.3967   | 0.8414    | 0.04051   | 3.488     | 134.8     | 0.6537    |
| 9         | -0.4132   | 0.9623    | 0.1579    | 8.892     | 136.5     | 0.6551    |
| 10        | -0.4232   | 0.8663    | 0.08823   | 9.86

In [21]:
# Use all features
X = df_lagged[selected_feature_sets['All Features']].values

results_xgb_val = []

# Best parameters from Bayesian optimization
best_params = {
    'colsample_bytree': 0.9492450146042779,
    'learning_rate': 0.06883722343102901,
    'max_depth': int(round(2.653260334773832)),
    'n_estimators': int(round(138.51179986420755)),
    'subsample': 0.5282126033716135,
    'random_state': 42,
    'verbosity': 0
}

for i in range(initial_train_size, len(df_lagged) - forecast_horizon + 1, step_size):
    X_total_train = X[:i]
    y_total_train = y[:i]
    X_test = X[i:i + forecast_horizon]
    y_test = y[i:i + forecast_horizon]

    val_size = int(0.1 * len(X_total_train))
    if val_size < 1:
        continue

    X_sub_train = X_total_train[:-val_size]
    y_sub_train = y_total_train[:-val_size]
    X_val = X_total_train[-val_size:]
    y_val = y_total_train[-val_size:]

    model = xgb.XGBRegressor(**best_params)
    model.fit(X_sub_train, y_sub_train)
    y_pred_val = model.predict(X_val)

    mae = mean_absolute_error(y_val, y_pred_val)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred_val))
    r2 = r2_score(y_val, y_pred_val)
    mape = np.mean(np.abs((y_val - y_pred_val) / y_val)) * 100

    results_xgb_val.append({
        'Window': i,
        'MAE': mae,
        'RMSE': rmse,
        'R²': r2,
        'MAPE': mape
    })

results_xgb_df = pd.DataFrame(results_xgb_val)
print(results_xgb_df)
print("Average MAPE:", results_xgb_df['MAPE'].mean())


c:\Users\dylan\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
c:\Users\dylan\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
c:\Users\dylan\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


    Window          MAE          RMSE         R²       MAPE
0       12    91.630005     91.630004        NaN  25.173078
1       15    29.096344     29.096344        NaN   9.763874
2       18    55.359192     55.359192        NaN  27.405541
3       21    85.549675     87.930620 -17.398083  31.702155
4       24   130.578171    151.168418 -10.539902  26.086246
..     ...          ...           ...        ...        ...
61     195  5600.267090   7044.934350  -1.891419  25.675304
62     198  4864.633301   5917.973640  -1.043551  21.998657
63     201  6016.704102   7167.820031  -1.747392  25.571027
64     204  8183.595215   9310.909300  -4.793980  33.165758
65     207  9739.741211  10548.643135 -10.139050  38.628281

[66 rows x 5 columns]
Average MAPE: 27.622186063109186
